# pyBeamprofiler Demo

**Real-time laser beam profiling with Gaussian fitting**

This notebook demonstrates the `pybeamprofiler` package capabilities:
- Single-shot and continuous beam acquisition
- 1D/2D Gaussian fitting with multiple width definitions
- Interactive camera controls (Jupyter Widgets)
- Real-time visualization with beam overlay
- High performance (513+ fps fitting, 15-30 Hz display)

**Performance modes:**
- Standard: Full display with profiles (~15-20 Hz)
- Fast: Heatmap only for maximum speed (~25-30 Hz)

Using simulated camera for demonstration (larger beam spot for better visibility).  
Supports FLIR/Basler cameras via GenICam for real hardware.

In [1]:
# Import the beam profiler (installed in editable mode with `pip install -e .`)
from pybeamprofiler import BeamProfiler

In [2]:
# List available GenICam cameras (FLIR, Basler, etc.)
# This will search for installed GenTL producers and list connected cameras
from pybeamprofiler import print_camera_info

print("Searching for GenICam cameras...")
print_camera_info()  # Lists all found cameras with vendor, model, serial number

Searching for GenICam cameras...
No cameras found.

Make sure:
  1. Camera is connected
  2. GenTL producer (.cti) is installed:
     - FLIR: Spinnaker SDK
     - Basler: Pylon SDK


## Camera Setup (Optional)

For real FLIR or Basler cameras, set the `GENICAM_GENTL64_PATH` environment variable:

**Linux/macOS:**
```bash
export GENICAM_GENTL64_PATH=/opt/spinnaker/lib/flir-gentl
# or for Basler
export GENICAM_GENTL64_PATH=/opt/pylon/lib64/gentlproducer/gtl
```

**Windows:**
```cmd
set GENICAM_GENTL64_PATH=C:\Program Files\FLIR Systems\Spinnaker\cti64\vs2015
```

This is the standard GenICam way - once set, cameras are auto-discovered!

In [3]:
# Initialize beam profiler with simulated camera
# For real cameras, use: camera="flir" or camera="basler"
bp = BeamProfiler(camera="simulated")

print(f"Initialized with {type(bp._camera).__name__}")
print(f"Sensor: {bp.width_pixels}×{bp.height_pixels} pixels, {bp.pixel_size:.2f} um/pixel")

Initialized with SimulatedCamera
Sensor: 1024×1024 pixels, 5.00 um/pixel


In [4]:
# Display interactive camera settings with enhanced Jupyter Widgets
# Features: Tabs (Camera Settings | Acquisition), Accordions, Dual inputs (slider + text)
bp.setting()

In [5]:
# Single-shot acquisition with 1D Gaussian fitting
# Fits projections along X and Y axes (fastest method, 850+ fps)
bp.fit_method = '1d'
bp.plot(num_img=1, exposure_time=0.01)  # 10ms exposure, 1 image

print(f"Measured Beam Width: {bp.width:.2f} um")

Measured Beam Width: 2084.94 um


In [6]:
# Single-shot acquisition with 2D Gaussian fitting
# Fits full 2D Gaussian with rotation angle (more accurate for elliptical beams)
bp.fit_method = '2d'
bp.plot(num_img=1, exposure_time=0.01)  # 10ms exposure, 1 image

print(f"Beam Width: {bp.width:.2f} um, Angle: {bp.angle_deg:.1f} degrees")

Beam Width: 2026.21 um, Angle: 180.0 degrees


In [7]:
# Continuous acquisition with live streaming and fitting
# Uses IPython's clear_output + display for live updates in Jupyter
#
# Performance:
#   - Gaussian fitting: 850+ fps (< 2ms per frame) - NOT the bottleneck!
#   - Display rendering: ~110ms per frame (this is the bottleneck)
#   - Actual update rate: ~6-10 Hz (limited by Jupyter display rendering)
#
# What you'll see:
#   - Plot refreshes continuously with NEW beam image
#   - Frame counter increments: "Frame #1", "Frame #2", etc.
#   - Width/center values change each frame (beam fluctuation)
#   - FPS counter shows actual update rate (~6-10 fps)
#   - Every frame has fresh Gaussian fit!
#
# To stop: Press Jupyter's interrupt button or Kernel -> Interrupt

bp.fit_method = '1d'  # Use 1D for fastest fitting (850+ fps)
bp.plot()  # Standard mode: Full display with profiles (~6-10 Hz)


Live stream stopped after 30 frames (9.8 fps average)


In [ ]:
# FAST mode: Heatmap only for better performance
# Shows only the beam image with ellipse overlay (no profile plots)
# Slightly faster since less data to render

bp.fit_method = '1d'
bp.plot(heatmap_only=True)  # Fast mode: Heatmap only (~8-12 Hz)